# Spatial Resolution & Swath vs Focal Length Trade
Tradeoff between spatial resolution and swath width for a given range of foreoptic focal lengths.

In [7]:
# stdlib
from pathlib import Path

# external
import astropy.units as unit
import numpy as np
import pandas as pd
import plot
from IPython.display import display

# project
from architect import components, systems
from architect.libs import utillib
from architect.systems.optical import foreoptics, masks, sensors, spectrometers

## Setup

In [8]:
output_dir = Path("output/spatial_resolution_&_swath_vs_focal_length_trade/")
output_dir.mkdir(parents=True, exist_ok=True)

### Parameters

In [9]:
# constants
orbital_altitude = 550 * unit.km
foreoptic_diameter = 44 * unit.mm
skew_angle = np.array([40, 40]) * unit.deg
slit_size = np.array([3, 1]) * unit.mm
target_wavelength = 1300 * unit.nm

# variables
focal_length = np.arange(start=10, stop=300, step=10) * unit.mm

# vectorization
parameter_space_shape = (focal_length.size, slit_size.size)
focal_length = utillib.orient_and_broadcast(
    a=focal_length, dim=0, shape=parameter_space_shape
)

### Instantiation

In [10]:
# components
sensor = sensors.TauSWIR()

foreoptic = foreoptics.Foreoptic(diameter=foreoptic_diameter, focal_length=focal_length)

slit = masks.RectSlit(size=slit_size)

# systems
payload = spectrometers.HyperspectralImager(
    sensor=sensor, foreoptic=foreoptic, slit=slit
)

## Pipeline

In [11]:
sensor_spatial_resolution = payload.get_spatial_resolution(
    wavelength=target_wavelength,
    target_distance=orbital_altitude,
    skew_angle=skew_angle[0],
).to(unit.m)

display(
    f"Sensor-limited spatial resolution test shape: {sensor_spatial_resolution.shape}"
)

swath = payload.get_swath(altitude=orbital_altitude, skew_angle=skew_angle)
display(f"Swath shape: {swath.shape}")

'Sensor-limited spatial resolution test shape: (29, 2)'

'Swath shape: (29, 2)'

## Plots

In [12]:
focal_length_label = f"Focal Length [{focal_length.unit}]"
sensor_spatial_resolution_label = (
    f"Spatial Resolution [{sensor_spatial_resolution.unit}]"
)
swath_label_x = f"Swath (across-track) [{swath.unit}]"
swath_label_y = f"Swath (along-track) [{swath.unit}]"


data = {
    focal_length_label: focal_length[:, 0],
    sensor_spatial_resolution_label: sensor_spatial_resolution[:, 0],
    swath_label_x: swath[:, 0],
    swath_label_y: swath[:, 1],
}

df = pd.DataFrame.from_dict(data=data)
fig = plot.line(
    df=df,
    x=focal_length_label,
    y=[
        sensor_spatial_resolution_label,
        swath_label_x,
        swath_label_y,
    ],
    title=None,
    dark=True,
)

fig.show()
display(df)
display(payload)

,Focal Length [mm],Spatial Resolution [m],Swath (across-track) [km],Swath (along-track) [km]
0,10.0,825.000000,285.700616,93.890118
1,20.0,412.500000,141.146285,46.883056
2,30.0,275.000000,93.890118,31.247728
3,40.0,206.250000,70.363306,23.433791
4,50.0,165.000000,56.270568,18.746290
5,60.0,137.500000,46.883056,15.621572
6,70.0,117.857143,40.180784,13.389745
7,80.0,103.125000,35.155521,11.715929
8,90.0,91.666667,31.247728,10.414099
9,100.0,82.500000,28.121910,9.372650


,Value,Units
foreoptic,Foreoptic,None
slit,RectSlit,None
diffractor,None,None
sensor,TauSWIR,None
systems,list [4],None
dimensions,None,None
mass,None,None
volume,None,None
density,None,None
index,None,None
